In [ ]:
from torchsummary import summary

### Kaggle dataset loader set

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 398, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation

Dataset URL: https://www.kaggle.com/datasets/mateuszbuda/lgg-mri-segmentation
License(s): CC-BY-NC-SA-4.0
 99% 708M/714M [00:06<00:00, 109MB/s]
100% 714M/714M [00:06<00:00, 110MB/s]


In [ ]:
!unzip lgg-mri-segmentation.zip

Streaming output truncated to the last 5000 lines.
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7294_19890104/TCGA_DU_7294_19890104_9_mask.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_1.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_10.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_10_mask.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_11.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_11_mask.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_12.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_12_mask.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_19910324/TCGA_DU_7298_19910324_13.tif  
  inflating: lgg-mri-segmentation/kaggle_3m/TCGA_DU_7298_1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
/content/kaggle_3m/TCGA_CS_4941_19960909

NameError: name 'content' is not defined

### Dataset

In [ ]:
import glob
# Make objects with the images and masks.
brain_scans = []
dataset_path = "/content/kaggle_3m/"
mask_files = glob.glob(dataset_path + '*/*_mask*')
# mask_files = glob.glob('../kaggle/lgg-mri-segmentation/kaggle_3m/*/*_mask*')
print(mask_files[0])
for i in mask_files:
    # print(i)
    brain_scans.append(i.replace('_mask',''))

print(brain_scans[:10])
print(mask_files[:10])

In [ ]:
import pandas as pd
img_df = pd.DataFrame({
    "image_path":brain_scans,
    "mask_path":mask_files
})

In [ ]:
import cv2
import numpy as np
def positive_negative_diagnosis(file_masks):
    mask = cv2.imread(file_masks)
    value = np.max(mask)
    if value > 0:
        return 1
    else:
        return 0

# Apply the function to the masks and return back a column with 1 and zeros, where 0 indicate no tumor and 1 a tumor.
img_df["Tumor"] = img_df["mask_path"].apply(lambda x: positive_negative_diagnosis(x))

In [ ]:
img_df

In [ ]:
img_df["Tumor"].value_counts()

In [ ]:
# import matplotlib.pyplot as plt

# for i in range(1,40, 2):
#     img_path=brain_scans[i]
#     msk_path=mask_files[i]
#     img=cv2.imread(img_path)
#     img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     msk=cv2.imread(msk_path)

#     #Plot the Brain MRI scans
#     original = img.copy()
#     fig, ax = plt.subplots(1,3,figsize = (15,5))
#     ax[0].imshow(original)
#     ax[0].set_title("Brain MRI")

#     # Plot the corresponding mask
#     main = original.copy()
#     mask = msk.copy()
#     ax[1].imshow(mask)
#     ax[1].set_title("Tumor Location")

#     # Plot the Brain MRI scan with their mask
#     main = original.copy()
#     label = cv2.imread(msk_path)
#     sample = np.array(np.squeeze(label), dtype = np.uint8)
#     contours, hier = cv2.findContours(sample[:,:,0],cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
#     sample_over_gt = cv2.drawContours(main, contours, -1,[255,0,0], thickness=-1)
#     ax[2].imshow(sample_over_gt)
#     ax[2].set_title("MRI Brain with highlighted Tumor")

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(img_df[:3000],test_size = 0.1)
train, val = train_test_split(train,test_size = 0.2)
print(train.values.shape)
print(val.values.shape)
print(test.values.shape)

train.head()
val.head()

In [ ]:
img_df['Tumor'] = img_df['Tumor'].apply(lambda x: str(x))
img_df.info()

### Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


def train_generator(data_frame, batch_size, aug_dict,
        image_color_mode="rgb",
        mask_color_mode="grayscale",
        image_save_prefix="image",
        mask_save_prefix="mask",
        save_to_dir=None,
        target_size=(128,128),
        seed=1):
    '''
    can generate image and mask at the same time use the same seed for
    image_datagen and mask_datagen to ensure the transformation for image
    and mask is the same if you want to visualize the results of generator,
    set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)

    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        x_col = "image_path",
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)

    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        x_col = "mask_path",
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)

    train_gen = zip(image_generator, mask_generator)

    for (img, mask) in train_gen:
        img, mask = adjust_data(img, mask)
        yield (img,mask)

def adjust_data(img,mask):
    img = img / 127
    mask = mask / 127
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0

    return (img, mask)

### Hyperparameters

In [ ]:
EPOCHS = 30
BATCH_SIZE = 32
im_height = 128
im_width = 128

In [ ]:
train_generator_args = dict(width_shift_range=0.05,
                            height_shift_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest'
                           )


train_gen = train_generator(train, BATCH_SIZE,
                            train_generator_args,
                            target_size=(im_height,
                            im_width)
                           )


test_gen = train_generator(val, BATCH_SIZE,
                            dict(),
                            target_size=(im_height, im_width)
                            )

### Loss Function

In [ ]:
from tensorflow import keras
import tensorflow.keras.backend as K
smooth=100

def dice_coef(y_true, y_pred):
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)
    And=K.sum(y_truef* y_predf)
    return((2* And + smooth) / (K.sum(y_truef) + K.sum(y_predf) + smooth))

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def jac_distance(y_true, y_pred):
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)

    return - iou(y_true, y_pred)

### Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        return x

class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.block = UNetBlock(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.block(x)
        p = self.pool(x)
        return x, p

class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpBlock, self).__init__()
        self.block = UNetBlock(in_channels, out_channels)
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([x, skip], dim=1)
        x = self.block(x)
        return x


In [ ]:
class DiffusionProcess:
    def __init__(self, beta_start, beta_end, num_steps):
        self.betas = torch.linspace(beta_start, beta_end, num_steps)

    def forward(self, x, t):
        noise = torch.randn_like(x)
        return x + self.betas[t] * noise

    def reverse(self, noisy_x, t):
        return noisy_x / (1 + self.betas[t])


In [ ]:
class DiffusionUNet(nn.Module):
    def __init__(self, in_channels, out_channels, beta_start=0.1, beta_end=0.2, num_steps=1000):
        super(DiffusionUNet, self).__init__()
        self.encoder1 = DownBlock(in_channels, 64)
        self.encoder2 = DownBlock(64, 128)
        self.encoder3 = DownBlock(128, 256)

        self.bottleneck = UNetBlock(256, 512)

        self.decoder1 = UpBlock(512, 256)
        self.decoder2 = UpBlock(256, 128)
        self.decoder3 = UpBlock(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

        self.diffusion = DiffusionProcess(beta_start, beta_end, num_steps)

    def forward(self, x, t=None):
        # Apply forward diffusion process
        if t is not None:
          noisy_x = self.diffusion.forward(x, t)
        else:
          noisy_x=x

        # Encoder
        enc1, p1 = self.encoder1(noisy_x)
        enc2, p2 = self.encoder2(p1)
        enc3, p3 = self.encoder3(p2)

        # Bottleneck
        bottleneck = self.bottleneck(p3)

        # Decoder
        dec1 = self.decoder1(bottleneck, enc3)
        dec2 = self.decoder2(dec1, enc2)
        dec3 = self.decoder3(dec2, enc1)

        # Final convolution
        out = self.final_conv(dec3)

        return out


In [ ]:
model = DiffusionUNet(in_channels=3, out_channels=3)
summary(model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
         UNetBlock-5         [-1, 64, 256, 256]               0
         MaxPool2d-6         [-1, 64, 128, 128]               0
         DownBlock-7  [[-1, 64, 256, 256], [-1, 64, 128, 128]]               0
            Conv2d-8        [-1, 128, 128, 128]          73,856
              ReLU-9        [-1, 128, 128, 128]               0
           Conv2d-10        [-1, 128, 128, 128]         147,584
             ReLU-11        [-1, 128, 128, 128]               0
        UNetBlock-12        [-1, 128, 128, 128]               0
        MaxPool2d-13          [-1, 128, 64, 64]               0
        DownBlock-14  [[

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=15,
                              min_delta=0.0001,
                              factor=0.2
                             )

# Define the earltstop function
earlystopping = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              mode='min',
                              verbose=1,
                              patience=20,
                              restore_best_weights = True
                             )

# Defining the Callbacks Function
checkpointer2 = [ModelCheckpoint(f'{save_dir}/base_model_unet.hdf5', verbose=1, save_best_only=True)]
callbacks = [checkpointer2, earlystopping, reduce_lr]

In [ ]:
def train(model, dataloader, epochs, diffusion):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        for data in dataloader:
            inputs, labels = data
            t = torch.randint(0, diffusion.num_steps, (1,)).item()
            noisy_inputs = diffusion.forward(inputs, t)
            outputs = model(noisy_inputs, t)
            loss = nn.MSELoss()(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

# Example usage
diffusion = DiffusionProcess(beta_start=0.1, beta_end=0.2, num_steps=1000)
model = DiffusionUNet(in_channels=3, out_channels=1)
# Assume dataloader is predefined
train(model, dataloader, epochs=10, diffusion=diffusion)